# Extract Lemmatization from JSON: Extended Parser
The code in this notebook will the parse [ORACC](http://oracc.org) `JSON` file of the Ur III corpus to extract lemmatization data. 

The output contains text IDs, line IDs, lemmas, and (potentially) other data. 

In [1]:
import pandas as pd
import zipfile
import json
import requests
from tqdm.auto import tqdm
import os
import errno

## 0 Create Directories, if Necessary
The two directories needed for this script are `jsonzip` and `output`.  

In [2]:
for d in ['jsonzip', 'output']:
    try:
        os.mkdir(d)
    except OSError as exc:
        if exc.errno !=errno.EEXIST:
            raise
        pass

## 1.1 Input Project Name
In order to download the data we need to know the [ORACC](http://oracc.org) project name of the Ur III data. This project name corresponds to the last part of the URL; the URL is http://oracc.org/epsd2/admin/u3adm and thus the project name is 'epsd2/admin/u3adm'.

In [3]:
p = ['epsd2/admin/u3adm']

## Download the ZIP files.
Download the zipped JSON files from the [ORACC](http://oracc.org) server.

In [4]:
CHUNK = 16 * 1024
for project in p:
    proj = project.replace('/', '-')
    url = "http://build-oracc.museum.upenn.edu/json/" + proj + ".zip"
    file = 'jsonzip/' + proj + '.zip'
    with requests.get(url, stream=True) as r:
        if r.status_code == 200:
            tqdm.write('Saving ' + url + ' as ' + file)
            with open(file, 'wb') as f:
                for c in tqdm(r.iter_content(chunk_size=CHUNK), desc = project):
                    f.write(c)
        else:
            tqdm.write("WARNING: " + url + " does not exist.")

Saving http://build-oracc.museum.upenn.edu/json/epsd2-admin-u3adm.zip as jsonzip/epsd2-admin-u3adm.zip


## <a name="head21"></a>2.1 The `parsejson()` function
The `parsejson()` function will "dig into" the `json` file (transformed into a dictionary) until it finds the relevant data. The `json` file consists of a hierarchy of `cdl` nodes; only the lowest nodes contain lemmatization data. The function goes down this hierarchy by calling itself when another `cdl` node is encountered. For more information about the data hierarchy in the [ORACC](http://oracc.org) `json` files, see [ORACC Open Data](http://oracc.org/doc/opendata/index.html).

The argument of the `parsejson()` function is a `JSON` object, essentially a Python dictionary that initially contains the entire contents of the original JSON file. The code takes the key `cdl` the value of which is a list of `JSON` dictionaries. Iterating through these dictionaries, if a dictionary contains another `cdl` node, the function calls itself with this lower-level dictionary as argument. This way the function digs deeper and deeper into the `JSON` tree, until it does not encounter a `cdl` key anymore. Here we are at the level of individual words. The code checks for a key `f`, if it exists the value of that key (another dictionary) is appended to the list `lemm_l`. The list `lemm_l`, which is initiated outside of the function proper, will become a list of dictionaries, where each dictionary represents a single word.

The variable `id_text` consists of a project abbreviation, such as `blms` or `cams/gkab` plus a text ID, in the format `cams/gkab/P338616` or `dcclt/Q000039`. The `id_text` is a global variable that is defined each time `parsejson()` is called in the main process. Therefore, it can be accessed from within the function and is added to the lemmatization data of every word.

The field `word_id` consists of three parts, namely a text ID, line ID, and word ID, in the format `Q000039.76.2` meaning: the second word in line 76 of text object `Q000039`. Note that `76` is not a line number strictly speaking but an object reference within the text object. Things like horizontal rulings, columns, and breaks also get object references. The `word_id` field allows us to put lines, breaks, and horizontal drawings together in the proper order.

The field `label` is a human-legible label that refers to a line or another part of the text; it may look like `o i 23` (obverse column 1 line 23) or `r v 23'` (reverse column 5 line 23 prime). The `label` field is used in online [ORACC](http://oracc.org) editions to indicate line numbers.

The fields `extent`, `scope`, and `state` give metatextual data about the condition of the object; they capture the number of broken lines or columns and similar information. 

In [5]:
def parsejson(text):
    for JSONobject in text["cdl"]:
        if "cdl" in JSONobject: 
            parsejson(JSONobject)
        if "label" in JSONobject:
            meta_d["label"] = JSONobject['label']
        if "f" in JSONobject:
            lemma = JSONobject["f"]
            lemma["id_word"] = JSONobject["ref"]
            lemma['label'] = meta_d["label"]
            lemma["id_text"] = meta_d["id_text"]
            lemm_l.append(lemma)
        if "strict" in JSONobject and JSONobject["strict"] == "1":
            lemma = {key: JSONobject[key] for key in dollar_keys}
            lemma["id_word"] = JSONobject["ref"]
            lemma["id_text"] = meta_d["id_text"]
            lemm_l.append(lemma)
    return

## 2.2 Call the `parsejson()` function for every `JSON` file
The code in this cell will iterate through the list of projects entered above (1.1). For each project the `JSON` zip file is located in the directory `jsonzip`, named PROJECT.zip. The `zip` file contains a directory that is called `corpusjson` that contains a JSON file for every text that is available in that corpus. The files are called after their text IDs in the pattern `P######.json` (or `Q######.json` or `X######.json`).

The function `namelist()` of the `zipfile` package is used to create a list of the names of all the files in the ZIP. From this list we select all the file names in the `corpusjson` directory with extension `.json` (this way we exclude the name of the directory itself). 

Each of these files is read from the `zip` file and loaded with the command `json.loads()`, which transforms the string into a proper JSON object. 

This JSON object (essentially a Python dictionary), which is called `data_json` is now sent to the `parsejson()` function. The function adds lemmata to the `lemm_l` list. In the end, `lemm_l` will contain as many list elements as there are words in all the texts in the projects requested.

The dictionary `meta_d` is created to hold temporary information. The value of the key `id_text` is updated in the main process every time a new JSON file is opened and send to the `parsejson()` function. The `parsejson()` function itself will change values or add new keys, depending on the information found while iterating through the JSON file. When a new lemma row is created, `parsejon()` will supply data such as `id_text`, `label` and (potentially) other information from `meta_d`.

In [6]:
lemm_l = []
meta_d = {"label": None, "id_text": None}
dollar_keys = ["extent", "scope", "state"]
for project in p:
    file = "jsonzip/" + project.replace("/", "-") + ".zip"
    try:
        z = zipfile.ZipFile(file)       # create a Zipfile object
    except:
        print(file + " does not exist or is not a proper ZIP file")
        continue
    files = z.namelist()     # list of all the files in the ZIP
    files = [name for name in files if "corpusjson" in name and name[-5:] == '.json']                                                                                                  #that holds all the P, Q, and X numbers.
    for filename in tqdm(files, desc = project):       #iterate over the file names
        id_text = project + filename[-13:-5] # id_text is, for instance, blms/P414332
        meta_d["id_text"] = id_text
        try:
            st = z.read(filename).decode('utf-8')         #read and decode the json file of one particular text
            data_json = json.loads(st)                # make it into a json object (essentially a dictionary)
            parsejson(data_json)               # and send to the parsejson() function
        except:
            print(id_text + ' is not available or not complete')
    z.close()

## 3 Data Structuring
### 3.1 Transform the Data into a DataFrame
The list `lemm_l` is transformed into a `pandas` dataframe for further manipulation.

In [9]:
words_df = pd.DataFrame(lemm_l)
words_df = words_df.fillna('')   # replace NaN (Not a Number) with empty string
words_df.head(10)

,aform,base,cf,cont,delim,epos,extent,form,gdl,gw,...,label,lang,morph,norm,norm0,pos,scope,sense,state,stem
0,,,,,,,,3(diš),"[{'n': 'n', 'form': '3(diš)', 'id': 'P124149.3...",,...,o 1,sux,,,,n,,,,
1,,udu,udu,,,N,,udu,"[{'v': 'udu', 'id': 'P124149.3.2.0'}]",sheep,...,o 1,sux,~,,udu,N,,sheep,,
2,,,,,,,,6(diš),"[{'n': 'n', 'form': '6(diš)', 'id': 'P124149.4...",,...,o 2,sux,,,,n,,,,
3,,u₈,u,,,N,,u₈,"[{'v': 'u₈', 'id': 'P124149.4.2.0'}]",ewe,...,o 2,sux,~,,u,N,,"sheep, ewe",,
4,,,,,,,,2(diš),"[{'n': 'n', 'form': '2(diš)', 'id': 'P124149.5...",,...,o 3,sux,,,,n,,,,
5,,maš₂-gal,mašgal,,,N,,maš₂-gal,"[{'v': 'maš₂', 'id': 'P124149.5.2.0', 'delim':...",goat,...,o 3,sux,~,,mašgal,N,,billy goat,,
6,,šu-gid₂,šugid,,,N,,šu-gid₂,"[{'v': 'šu', 'id': 'P124149.6.1.0', 'delim': '...",~animal,...,o 4,sux,~,,šugid,N,,"designation of animals, unfit for work?",,
7,,e₂-muhaldim,emuhaldim,,,N,,e₂-muhaldim,"[{'v': 'e₂', 'id': 'P124149.6.2.0', 'delim': '...",kitchen,...,o 4,sux,~,,emuhaldim,N,,kitchen,,
8,,mu,mu,,,N,,mu,"[{'v': 'mu', 'id': 'P124149.7.1.0'}]",name,...,o 5,sux,~,,mu,N,,name,,
9,,gar₃-du,gardu,,,N,,gar₃-du-e-ne-še₃,"[{'v': 'gar₃', 'id': 'P124149.7.2.0', 'delim':...",soldier,...,o 5,sux,"~,ene.eše",,"gardu,ene.eše",N,,soldier,,


## 3.2 Remove Spaces and Commas from Guide Word and Sense
Spaces and commas in Guide Word and Sense may cause trouble in computational methods in tokenization, or when saved in Comma Separated Values format. All spaces and commas are replaced by hyphens and nothing (empty string), respectively. By default the `replace()` function in `pandas` will match the entire string (that is, "lugal" matches "lugal" but there is no match between "l" and "lugal"). In order to match partial strings the parameter `regex` must be set to `True`.

The `replace()` function takes a nested dictionary as argument. The top-level keys identify the columns on which the `replace()` function should operate (in this case 'gw' and 'sense'). The value of each key is another dictionary with the search string as key and the replace string as value.

In [10]:
findreplace = {' ' : '-', ',' : ''}
words_df = words_df.replace({'gw' : findreplace, 'sense' : findreplace}, regex=True)

The columns in the resulting DataFrame correspond to the elements of a full [ORACC](http://oracc.org) signature, plus information about text, line, and word ids:
* base (Sumerian only)
* cf (Citation Form)
* cont (continuation of the base; Sumerian only)
* epos (Effective Part of Speech)
* form (transliteration, omitting all flags such as indication of breakage)
* frag (transliteration; including flags)
* gdl_utf8 (cuneiform)
* gw (Guide Word: main or first translation in standard dictionary)
* id_text (six-digit P, Q, or X number)
* id_word (word ID in the format Text_ID.Line_ID.Word_ID)
* label (traditional line number in the form o ii 2' (obverse column 2 line 2'), etc.)
* lang (language code, including sux, sux-x-emegir, sux-x-emesal, akk, akk-x-stdbab, etc)
* morph (Morphology; Sumerian only)
* norm (Normalization: Akkadian)
* norm0 (Normalization: Sumerian)
* pos (Part of Speech)
* sense (contextual meaning)
* sig (full ORACC signature)

Not all data elements (columns) are available for all words. Sumerian words never have a `norm`, Akkadian words do not have `norm0`, `base`, `cont`, or `morph`. Most data elements are only present when the word is lemmatized; only `lang`, `form`, `id_word`, and `id_text` should always be there.

## Create Line ID
The DataFrame currently has a word-by-word data representation. We will add to each word a field `id_line` that will make it possible to reconstruct lines. This newly created field `id_line` is different from a traditional line number (found in the field "label") in two ways. First, id_line is an integer, so that lines are sorted correctly. Second, `id_line` is assigned to words, but also to gaps and horizontal drawings on the tablet. The field `id_line` will allow us to keep all these elements in their proper order.

The field "id_line" is created by splitting the field "id_word" into (two or) three elements. The format of "id_word" is IDtext.line.word. The middle part, id_line, is selected and its data type is changed from string to integer. Rows that represent gaps in the text or horizontal drawings have an "id_word" in the format IDtext.line (consisting of only two elements), but are treated in exactly the same way.

In [13]:
words_df['id_line'] = [int(wordid.split('.')[1]) for wordid in words_df['id_word']]
words_df.head(10)

,aform,base,cf,cont,delim,epos,extent,form,gdl,gw,...,lang,morph,norm,norm0,pos,scope,sense,state,stem,id_line
0,,,,,,,,3(diš),"[{'n': 'n', 'form': '3(diš)', 'id': 'P124149.3...",,...,sux,,,,n,,,,,3
1,,udu,udu,,,N,,udu,"[{'v': 'udu', 'id': 'P124149.3.2.0'}]",sheep,...,sux,~,,udu,N,,sheep,,,3
2,,,,,,,,6(diš),"[{'n': 'n', 'form': '6(diš)', 'id': 'P124149.4...",,...,sux,,,,n,,,,,4
3,,u₈,u,,,N,,u₈,"[{'v': 'u₈', 'id': 'P124149.4.2.0'}]",ewe,...,sux,~,,u,N,,sheep-ewe,,,4
4,,,,,,,,2(diš),"[{'n': 'n', 'form': '2(diš)', 'id': 'P124149.5...",,...,sux,,,,n,,,,,5
5,,maš₂-gal,mašgal,,,N,,maš₂-gal,"[{'v': 'maš₂', 'id': 'P124149.5.2.0', 'delim':...",goat,...,sux,~,,mašgal,N,,billy-goat,,,5
6,,šu-gid₂,šugid,,,N,,šu-gid₂,"[{'v': 'šu', 'id': 'P124149.6.1.0', 'delim': '...",~animal,...,sux,~,,šugid,N,,designation-of-animals-unfit-for-work?,,,6
7,,e₂-muhaldim,emuhaldim,,,N,,e₂-muhaldim,"[{'v': 'e₂', 'id': 'P124149.6.2.0', 'delim': '...",kitchen,...,sux,~,,emuhaldim,N,,kitchen,,,6
8,,mu,mu,,,N,,mu,"[{'v': 'mu', 'id': 'P124149.7.1.0'}]",name,...,sux,~,,mu,N,,name,,,7
9,,gar₃-du,gardu,,,N,,gar₃-du-e-ne-še₃,"[{'v': 'gar₃', 'id': 'P124149.7.2.0', 'delim':...",soldier,...,sux,"~,ene.eše",,"gardu,ene.eše",N,,soldier,,,7


## 4 Create Lemma Column
A lemma, [ORACC](http://oracc.org) style, combines Citation Form, GuideWord and POS into a unique reference to one particular lemma in a standard dictionary, as in `lugal[king]N` (Sumerian) or `šarru[king]N`. 

Proper Nouns (proper names, geographical names, etc.) are a special case, because they currently receive a Part of Speech, but not a Citation Form or Guide Word.

Usually, not all words in a text are lemmatized, because a word may be (partly) broken and/or unknown. Unlemmatized and unlemmatizable words will receive a place-holder lemmatization that consists of the transliteration of the word (instead of the Citation Form), with `NA` as GuideWord and `NA` as POS, as in `i-bu-x[NA]NA`. Note that `NA` is a string. 

Finally, rows representing horizontal rulings, blank lines, or broken lines have the empty string in both Citation Form and Form. In those cases Lemma equals the English word 'break' in order to show that there is either a physical break or a logical break in the document.

In [16]:
proper_nouns = ['FN', 'PN', 'DN', 'AN', 'WN', 'ON', 'TN', 'MN', 'CN', 'GN']
words_df['lemma'] = words_df["cf"] + '[' + words_df["gw"] + ']' + words_df["pos"]
words_df.loc[words_df["cf"] == "" , 'lemma'] = words_df['form'] + '[NA]NA'
words_df.loc[words_df["pos"] == "n" , 'lemma'] = words_df['form'] + '[]NU'
words_df.loc[words_df["pos"].isin(proper_nouns) , 'lemma'] = words_df['form'] + '[]' + words_df['pos']
words_df.loc[words_df["form"] == "", 'lemma'] = "break"
words_df.head(10)

,aform,base,cf,cont,delim,epos,extent,form,gdl,gw,...,morph,norm,norm0,pos,scope,sense,state,stem,id_line,lemma
0,,,,,,,,3(diš),"[{'n': 'n', 'form': '3(diš)', 'id': 'P124149.3...",,...,,,,n,,,,,3,3(diš)[]NU
1,,udu,udu,,,N,,udu,"[{'v': 'udu', 'id': 'P124149.3.2.0'}]",sheep,...,~,,udu,N,,sheep,,,3,udu[sheep]N
2,,,,,,,,6(diš),"[{'n': 'n', 'form': '6(diš)', 'id': 'P124149.4...",,...,,,,n,,,,,4,6(diš)[]NU
3,,u₈,u,,,N,,u₈,"[{'v': 'u₈', 'id': 'P124149.4.2.0'}]",ewe,...,~,,u,N,,sheep-ewe,,,4,u[ewe]N
4,,,,,,,,2(diš),"[{'n': 'n', 'form': '2(diš)', 'id': 'P124149.5...",,...,,,,n,,,,,5,2(diš)[]NU
5,,maš₂-gal,mašgal,,,N,,maš₂-gal,"[{'v': 'maš₂', 'id': 'P124149.5.2.0', 'delim':...",goat,...,~,,mašgal,N,,billy-goat,,,5,mašgal[goat]N
6,,šu-gid₂,šugid,,,N,,šu-gid₂,"[{'v': 'šu', 'id': 'P124149.6.1.0', 'delim': '...",~animal,...,~,,šugid,N,,designation-of-animals-unfit-for-work?,,,6,šugid[~animal]N
7,,e₂-muhaldim,emuhaldim,,,N,,e₂-muhaldim,"[{'v': 'e₂', 'id': 'P124149.6.2.0', 'delim': '...",kitchen,...,~,,emuhaldim,N,,kitchen,,,6,emuhaldim[kitchen]N
8,,mu,mu,,,N,,mu,"[{'v': 'mu', 'id': 'P124149.7.1.0'}]",name,...,~,,mu,N,,name,,,7,mu[name]N
9,,gar₃-du,gardu,,,N,,gar₃-du-e-ne-še₃,"[{'v': 'gar₃', 'id': 'P124149.7.2.0', 'delim':...",soldier,...,"~,ene.eše",,"gardu,ene.eše",N,,soldier,,,7,gardu[soldier]N


# 5 Select Relevant Columns
Now we can select only the field 'lemma' and the fields that indicate the ID of the word, the line, or the document, plus the field 'label', which indicates the physical position of the line on the document.

In [18]:
cols = ['lemma', 'id_text', 'id_line', 'id_word', 'label']
words_df = words_df[cols].copy()
words_df.head(10)

,lemma,id_text,id_line,id_word,label
0,3(diš)[]NU,epsd2/admin/u3adm/P124149,3,P124149.3.1,o 1
1,udu[sheep]N,epsd2/admin/u3adm/P124149,3,P124149.3.2,o 1
2,6(diš)[]NU,epsd2/admin/u3adm/P124149,4,P124149.4.1,o 2
3,u[ewe]N,epsd2/admin/u3adm/P124149,4,P124149.4.2,o 2
4,2(diš)[]NU,epsd2/admin/u3adm/P124149,5,P124149.5.1,o 3
5,mašgal[goat]N,epsd2/admin/u3adm/P124149,5,P124149.5.2,o 3
6,šugid[~animal]N,epsd2/admin/u3adm/P124149,6,P124149.6.1,o 4
7,emuhaldim[kitchen]N,epsd2/admin/u3adm/P124149,6,P124149.6.2,o 4
8,mu[name]N,epsd2/admin/u3adm/P124149,7,P124149.7.1,o 5
9,gardu[soldier]N,epsd2/admin/u3adm/P124149,7,P124149.7.2,o 5


## 6 Save Results in CSV file or in Pickle
The output file is called `parsed.csv` and is placed in the directory `output`. In most computers, `csv` files open automatically in Excel. This program does not deal well with `utf-8` encoding (files in `utf-8` need to be imported; see the instructions [here](https://www.itg.ias.edu/content/how-import-csv-file-uses-utf-8-character-encoding-0). If you intend to use the file in Excel, change `encoding ='utf-8'` to `encoding='utf-16'`. For usage in computational text analysis applications `utf-8` is usually preferred. 

The Pandas function `to_pickle()` writes a binary file that can be opened in a later phase of the project with the `read_pickle()` command and will reproduce exactly the same DataFrame with the same data structure. The resulting file cannot be used in other programs.

In [19]:
savefile =  'parsed.csv'
with open('output/' + savefile, 'w', encoding="utf-8") as w:
    words_df.to_csv(w, index=False)
pickled = "output/parsed.p"
words_df.to_pickle(pickled)